<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Ensemble_with_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ensemble


In [1]:
'''imports '''
from google.colab import files
uploaded = files.upload()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from tensorflow.keras.models import Model, load_model


from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%pip install fasttext


# The complete datasets needed
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)


# The logits from the base model based on the descriptions, on the training and dev set, available in the datasets folder
# on github
#logits_train_model_description = np.load("train_logits_description.npy")
#logits_train_model_features = np.load("train_logits.npy")

# The logits from the base mode based on the extracted features, on the training and dev set , available in datasets folder
# on the github
#logits_dev_model_descriptions = np.load("dev_logits_description.npy")
#logits_dev_model_features = np.load("dev_logits.npy")

Saving dev_df_complete.json to dev_df_complete.json
Saving train_df_complete.json to train_df_complete.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313487 sha256=82a06178357d9ac63b9563593635f08ab225c438726abd648ec86d6111e1724c
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


Import the 2 models

In [17]:
!pip install huggingface_hub tensorflow --quiet

from huggingface_hub import login, create_repo, upload_folder
import os
import tensorflow as tf
from huggingface_hub import hf_hub_download

# Repo name and model file name
repo_id = "tgarnier067/MNLP-HW1"
model_name_1 = "model_logits_features.keras"
model_name_2 = "model_logits_description.keras"

# Download the model from Hugging Face
model_path_1 = hf_hub_download(repo_id=repo_id, filename=model_name_1)
model_path_2 = hf_hub_download(repo_id=repo_id, filename=model_name_2)

# Load the .keras model with TensorFlow
model_logits_features = tf.keras.models.load_model(model_path_1)
model_logits_description = tf.keras.models.load_model(model_path_2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_logits_features.keras:   0%|          | 0.00/8.74M [00:00<?, ?B/s]

model_logits_description.keras:   0%|          | 0.00/9.86M [00:00<?, ?B/s]

Prepare the data, on wich we will apply model_logits_description

In [3]:
 import fasttext.util
 fasttext.util.download_model('en', if_exists='ignore')  # English
 ft = fasttext.load_model('cc.en.300.bin')

In [4]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
from sklearn.preprocessing import normalize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
'''transforming the descriptions to the embeddings space , adding the oov information'''
def transform_description(description):
  oov_words = 0
  description = nltk.word_tokenize(description)
  description = [word for word in description if word not in stop_words]
  description = [word.lower() for word in description]
  for word in description:
    if word not in ft.words:
      oov_words += 1
  description = " ".join(description)
  description_vector = ft.get_sentence_vector(description)
  description_vector = normalize(description_vector.reshape(1, -1), norm='l2')[0]
  description_vector = np.append(description_vector, oov_words)
  return description_vector

In [6]:
train_texts = train_df['description'].to_list()
train_labels = train_df['label_int'].to_list()

# Exctract the text and the label : Dev
dev_texts = dev_df['description'].to_list()
dev_labels = dev_df['label_int'].to_list()

# Vectorize the texts with fasttext
train_vectors = np.array([transform_description(text) for text in train_texts])
dev_vectors = np.array([transform_description(text) for text in dev_texts])

In [7]:
logits_dev_model_descriptions = model_logits_description.predict(dev_vectors)
logits_train_model_description = model_logits_description.predict(train_vectors)

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 183ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [8]:
df_train_description = pd.DataFrame(logits_train_model_description, columns=['logit_0', 'logit_1', 'logit_2'])
df_train_description['label'] = train_df['label_int']
dev_df_description = pd.DataFrame(logits_dev_model_descriptions, columns=['logit_0', 'logit_1', 'logit_2'])
dev_df_description['label'] = dev_df['label_int']


In [9]:
'''sanity check on the dataframe'''
df_train_description.head(10)

,logit_0,logit_1,logit_2,label
0,1.905099,1.534088,-3.094239,0
1,0.760209,2.901672,-5.288947,1
2,-0.636569,2.368679,-4.468928,1
3,1.309631,1.982280,-3.745397,1
4,-1.129772,2.629756,-4.753025,1
5,1.146501,1.890622,-3.682914,1
6,0.381403,1.764993,-3.796714,0
7,1.686283,2.372820,-4.080890,1
8,-1.380256,2.684189,-4.805523,1
9,1.886965,1.921860,-3.410134,0


Prepare the data, on wich we will apply model_logits_features

In [10]:
# Get the list of explicatives variables
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

# Create the X and y, train and dev
X_train = train_df[list_var]
y_train = train_df['label_int'].values

X_dev = dev_df[list_var]
y_dev = dev_df['label_int'].values

# Selecting numerical columns from X_train (all of them are int64 or float64)
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Defining pipeline: impute Nan, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Appling pipeline to numeric columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols)
])


#adjustin the data to proper format
X_train_processed = preprocessor.fit_transform(X_train)
X_dev_processed = preprocessor.transform(X_dev)

# float cast
X_train_processed = X_train_processed.astype('float32')
X_dev_processed = X_dev_processed.astype('float32')

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_dev_cat = to_categorical(y_dev, num_classes=3)

In [11]:
logits_dev_model_features = model_logits_features.predict(X_dev_processed)
logits_train_model_features = model_logits_features.predict(X_train_processed)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


## The ensemble nn working on the logits of the trained two base nns

here we try to combine two classifiers, one basing on the neural network working on features extracted frrom the wikidata/wikipedia, and the second one basing on the neural network working on the embedding representation of the description of the wikipedia/wikidata entities. We combine them with a meta lerner that as training set  takes the logits for each label from both of the trained base neural networks, achieved on the train set. And the validation are the logits of these two networks acquired by predcition on the validation set.

In [14]:


early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = np.concatenate((logits_train_model_description, logits_train_model_features), axis=1)
dev_vectors = np.concatenate((logits_dev_model_descriptions, logits_dev_model_features), axis=1)




model_Ensemble = models.Sequential()


input_layer = layers.Input(shape=(6,), name='input_layer')
model_Ensemble.add(input_layer)

for units in [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32]:
    dense_layer = layers.Dense(units, activation='relu')
    relu_layer = LeakyReLU(alpha=0.1)
    batch_norm_layer = layers.BatchNormalization()
    dropout_layer = layers.Dropout(0.3 if units > 128 else 0.2)
    model_Ensemble.add(dense_layer)
    model_Ensemble.add(relu_layer)
    model_Ensemble.add(batch_norm_layer)
    model_Ensemble.add(dropout_layer)

model_Ensemble.add(layers.Dense(3, activation='softmax'))


model_Ensemble.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])




history = model_Ensemble.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 109ms/step - accuracy: 0.5203 - loss: 1.1940 - val_accuracy: 0.6400 - val_loss: 0.7444 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8526 - loss: 0.4412 - val_accuracy: 0.6533 - val_loss: 0.9718 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8696 - loss: 0.3872 - val_accuracy: 0.6967 - val_loss: 0.9474 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8811 - loss: 0.3336 - val_accuracy: 0.6933 - val_loss: 0.9150 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8849 - loss: 0.3357 - val_accuracy: 0.7100 - val_loss: 0.9767 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8877 - loss: 0.3217 - val_accuracy: 0.7200 - val_loss: 0.9818 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8902 - loss: 0.3100 - val_

### results

Seems overfitted to the training set, with discreancy of the accuracies between the training and the validation set.

In [15]:
y_train_pred = model_Ensemble.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_Ensemble.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Train accuracy : 91.07%
Dev accuracy   : 72.67%
